<a href="https://colab.research.google.com/github/farhanhubble/discover-drl/blob/master/Rediscovering_RL_Notebook_0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
agent = '⚽'
opponent = '👕'
goal = '🥅'

arena = [['⚽', ' ' , '👕', ' ' ],
         [' ' , ' ' , ' ' , '👕'],
         [' ' , '👕', ' ' , ' ' ],
         [' ' , ' ' , ' ' , '👕'],
         [' ' , '👕', ' ' , '🥅']]

In [ ]:
import numpy as np

class foolsball(object):

  def __to_state__(self,row,col):
    return row*self.n_cols + col

  def __to_indices__(self, state):
    row = state / self.n_rows
    col = state % self.n_rows
    return row,col

  def __deserialize__(self,map:list,agent:str,opponent:str, goal:str):
    self.n_rows = len(map)
    self.n_cols = len(map[0])
    self.n_states = self.n_rows * self.n_cols 
    self.map = np.asarray(map)

    self.state = None
    self.goal = None
    self.opponents = []

    for row in range(self.n_rows):
      for col in range(self.n_cols):
        if map[row][col] == agent:
          self.state = self.__to_state__(row,col)
        
        elif map[row][col] == agent:
          self.opponents.append(self.__to_state__(row,col))

        elif map[row][col] == goal:
          self.goal = self.__to_state__(row,col)

    assert self.state is not None, print(f"Map {map} does not specify an agent {agent} location")
    assert self.goal is not None,  print(f"Map {map} does not specify a goal {goal} location")
    assert empty(self.opponents),  print(f"Map {map} does not specify any opponents {opponent} location")




  def __init__(self,map,agent,opponent,goal):
    self.__deserialize__(map,agent,opponent,goal)
    self.actions = ['n','e','w','s']

  def reset():
